 🗒️ This notebook is divided into the following sections:

1. Fetch Feature Groups
2. Define Transformation functions
4. Create Feature Views
5. Create Training Dataset with training, validation and test splits

In [ ]:
!pip install -U hopsworks 

In [ ]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5315
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 2
)
weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 2
)

In [ ]:
query = air_quality_fg.select_all().join(weather_fg.select_all())
query.read()

,date,pm25,pm10,o3,no2,so2,co,aqi,winddir
0,1671494400000,46.0,0.0,0.0,0.0,0.0,0.0,46.0,193.6
1,1671408000000,60.0,16.0,13.0,19.0,0.0,0.0,60.0,180.8
2,1671321600000,103.0,28.0,14.0,29.0,0.0,0.0,103.0,139.5
3,1671235200000,93.0,44.0,4.0,24.0,0.0,0.0,93.0,121.6
4,1671148800000,85.0,42.0,10.0,31.0,0.0,0.0,85.0,19.6
...,...,...,...,...,...,...,...,...,...
349,1641340800000,37.0,31.0,17.0,25.0,0.0,0.0,37.0,291.2
350,1641254400000,26.0,22.0,19.0,19.0,0.0,0.0,26.0,280.8
351,1641168000000,38.0,12.0,24.0,23.0,0.0,0.0,38.0,224.0
352,1641081600000,49.0,23.0,27.0,19.0,0.0,0.0,49.0,221.0


--- 

## <span style="color:#ff5f27;"> 🖍 Feature View Creation and Retrieving </span>

In [ ]:
# no need : query = air_quality_fg.select_all().join(weather_fg.select_all())
query_show = query.show(5)
col_names = query_show.columns

query_show

,date,pm25,pm10,o3,no2,so2,co,aqi,winddir
0,1671494400000,46.0,0.0,0.0,0.0,0.0,0.0,46.0,193.6
1,1671408000000,60.0,16.0,13.0,19.0,0.0,0.0,60.0,180.8
2,1671321600000,103.0,28.0,14.0,29.0,0.0,0.0,103.0,139.5
3,1671235200000,93.0,44.0,4.0,24.0,0.0,0.0,93.0,121.6
4,1671148800000,85.0,42.0,10.0,31.0,0.0,0.0,85.0,19.6


### <span style="color:#ff5f27;">🧑🏻‍🔬 Transformation functions</span>

Hopsworks Feature Store provides functionality to attach transformation functions to training datasets.

Hopsworks Feature Store also comes with built-in transformation functions such as `min_max_scaler`, `standard_scaler`, `robust_scaler` and `label_encoder`.

In [ ]:
[t_func.name for t_func in fs.get_transformation_functions()]

['min_max_scaler', 'standard_scaler', 'robust_scaler', 'label_encoder']

You can retrieve transformation function you need.

To attach transformation function to training dataset provide transformation functions as dict, where key is feature name and value is online transformation function name.

Also training dataset must be created from the Query object. Once attached transformation function will be applied on whenever save, insert and get_serving_vector methods are called on training dataset object.

In [ ]:
category_cols = ['date','conditions','aqi']

mapping_transformers = {col_name:fs.get_transformation_function(name='standard_scaler') for col_name in col_names if col_name not in category_cols}
category_cols = {col_name:fs.get_transformation_function(name='label_encoder') for col_name in category_cols if col_name not in ['date','aqi']}

mapping_transformers.update(category_cols)

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.create_feature_view()` method.

You can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [ ]:
feature_view = fs.create_feature_view(
    name = 'air_quality_fv',
    version = 1,
    transformation_functions = mapping_transformers,
    query = query # Merg of two feature groups
)

For now `Feature View` is saved in Hopsworks and you can retrieve it using `FeatureStore.get_feature_view()`.

In [ ]:
feature_view = fs.get_feature_view(
    name = 'air_quality_fv',
    version = 1
)

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset you use `FeatureView.create_training_data()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- You can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

In [ ]:
#Maryam
# Create training datasets based event time filter
td, td_job = feature_view.create_training_data(
        # start_time="20210101",
        #end_time="20220228",    
        description='Air polution prediction training dataset ',
        data_format="csv",
        splits={'train': 0.8, 'test': 0.1, 'validate': 0.1},
        #coalesce=True,
        write_options={'wait_for_job': False},
    )
td(0)[2]